# Random Histogram Forest

In [1]:
import scipy.io as sio
import scipy.stats as sstats
import random 
import numpy as np
import kurtosis_sum as ks_cy
from timeit import timeit

In [2]:
# set the number of trees and max height
H = 5
T = 100

### Definition of Node and Leaf classes

In [3]:
# a Node has contains a splitting value, splitting attribute as well as left and right trees
class Node:
    def __init__(self, value, attribute, left, right):
        self.value = value
        self.attribute = attribute
        self.left = left
        self.right = right

    # prints the data and splitting information in order of left branch, parent node, right branch
    def printNode(self, level):
        tabs = ""
        for i in range(0, level):
            tabs += "\t"
        # Leaf case
        if (self.left is None and self.right is None):
            print(tabs + str(self.data))
        else:
            # taking into account splits that have all instances on one side
            if self.left is not None:
                Node.printNode(self.left, level+1)
            print(tabs + "a" + str(self.attribute) + " < " + str(self.value))
            if self.right is not None:
                Node.printNode(self.right, level+1)

# a Leaf is a Node that has all parameters set to None except for data which contains its instances
class Leaf(Node):
    def __init__(self, data):
        super().__init__(value=None, attribute=None, left=None, right=None)
        self.data = data

### Splitting functions

In [4]:
# use kurtosis sum to get best attribute for the split
# returns attribute, its column and a random split value
def get_attribute(X, r):
    for a in range(0, X.shape[1]):
        if ks_cy.kurtosis_sum(X, a) > r:
            a_col = X[:, a]
                        
            # ensures that the split will be proper (no split on extremes)
            a_val = np.amin(a_col)

            while a_val == np.amin(a_col) or a_val == np.max(a_col):
                a_val = random.uniform(np.amin(a_col), np.amax(a_col))

            return a, a_col, a_val

### Core algorithms

In [5]:
# construction of a random histogram tree
def rht(X, nd, h):
    # last condition checks if all instances are the same --> split not possible
    if nd >= h or X.shape[0] == 1 or np.unique(X, axis=0).size == 1:
        # returns instances without duplicates
        return Leaf(np.unique(X, axis=0))
    else:
        # attribute selected according to kurtosis
        ks = ks_cy.kurtosis_sum(X, X.shape[1]-1)
        # ks may not be included depending on rounding
        r = random.uniform(0, ks)
        
        a, a_col, a_val = get_attribute(X, r)
             
        Xl = X[X[:, a] < a_val]
        Xr = X[X[:, a] >= a_val]

        Xl = rht(Xl, nd + 1, h)
        Xr = rht(Xr, nd + 1, h)

        return Node(a_val, a, Xl, Xr)

# construction of a random histogram forest
def rhf(X, t, nd, h):
    # create an empty forest
    rhf = np.empty([t], dtype=object)

    # append t random histogram trees
    for i in range(t):
        rhf[i] = rht(X, nd, h)

    return rhf

### Anomaly scoring

In [6]:
def anomaly_score(rhf, n, x):
    sum = 0
    
    for i in range(0, rhf.size):
        # number of distinct instances in the leaf of the given instance
        leaf_size = find_instance(rhf[i], x).data.size
        p = leaf_size / n
        sum += np.log(1 / p)
        
    return sum

# returns Leaf of instance in a RHT
def find_instance(rht, x):
    if rht.left != None or rht.right != None:
        # not leaf
        a = rht.attribute
        a_val = rht.value
        
        if (x[a] < a_val):
            return find_instance(rht.left, x)
        else: 
            return find_instance(rht.right, x)
    else:
        # leaf
        if x in rht.data:
            return rht

### Unit tests

### Testing

In [7]:
# extract matlab data from dataset in the form of a dictionary of numpy arrays
# X is the data, y are the labels
mat_contents = sio.loadmat("satellite.mat")

#dataset = mat_contents['X'] 
#labels = mat_contents['y']
dataset = np.array([[1],[70],[80],[90],[100],[110],[111],[112],[113],[114],[115],[116]])

# print the first random histogram tree for the above dataset 
#Node.printNode(rhf(X=dataset, t=T, nd=0, h=H)[0])
#Node.printNode(rht(X=dataset, nd=0, h=H))
test_rhf = rhf(X=dataset, t=T, nd=0, h=H)


Node.printNode(test_rhf[0], 0)

'''scores = np.array([dataset.shape[0], 2])
for i, x in enumerate(dataset):
    score = anomaly_score(test_rhf, dataset.size, x)
    np.append(scores, [score, labels[i]])
    print("score = " + str(score) + " | anomaly? " + str(labels[i]))
    '''
    


		[[1]]
	a0 < 3.3885236864068413
		[[70]]
a0 < 76.49813462170493
			[[80]]
		a0 < 84.79090975838015
			[[90]]
	a0 < 92.49717338463756
			[[100]]
		a0 < 104.80524376383913
					[[110]
 [111]]
				a0 < 111.79283337880022
					[[112]]
			a0 < 112.60269423464402
					[[113]
 [114]]
				a0 < 114.23644256203144
					[[115]
 [116]]


'scores = np.array([dataset.shape[0], 2])\nfor i, x in enumerate(dataset):\n    score = anomaly_score(test_rhf, dataset.size, x)\n    np.append(scores, [score, labels[i]])\n    print("score = " + str(score) + " | anomaly? " + str(labels[i]))\n    '

### Testing Python vs Cython speed

In [8]:
# sum of log(Kurtosis(X[a] + 1)) of attributes 0 to d inclusive
def kurtosis_sum(X, d):
    sum = 0
    
    # loop over the transpose matrix in order to analyze by column
    for a in range(0, d+1):
        # + 4 since the scipy function for kurtosis subtracts 3 so +1 +3 = 4
        sum += np.log(sstats.stats.kurtosis(X[:,a])+4)
        
    return sum

In [9]:
# extract matlab data from dataset in the form of a dictionary of numpy arrays
# X is the data, y are the labels
mat_contents = sio.loadmat("satellite.mat")
test_data = mat_contents['X'] 

In [12]:
# Python 

code = '''kurtosis_sum(test_data, test_data.shape[1]-1)'''
kurtosis_sum(test_data, test_data.shape[1]-1)

#print("Total time for kurtosis sum (python) = ", timeit(stmt=code, number=100))



47.71513186362433

In [14]:
# Cython 
code = '''ks_cy.kurtosis_sum(test_data, test_data.shape[1]-1)'''

print("Total time for kurtosis sum (cython) = ", timeit(stmt=code, number=100))

NameError: name 'ks_cy' is not defined